# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [0]:
data = spark.sql("SELECT * FROM movielens_ratings_csv")

In [0]:
data.head()

Out[4]: Row(movieId=2, rating=3.0, userId=0)

In [0]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [0]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [0]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [0]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [0]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|    28|  1.1085978|
|      3|   1.0|    26| 0.59723073|
|      4|   4.0|    26|  1.8824284|
|      4|   1.0|    12|0.017507032|
|      6|   2.0|    22|   2.451865|
|      2|   2.0|    20|  1.5717993|
|      4|   1.0|     5| -0.4776708|
|      2|   1.0|    15|  1.8811086|
|      5|   2.0|    15|   1.380704|
|      2|   1.0|    17|  0.9150833|
|      0|   1.0|    23| 0.98612916|
|      0|   3.0|    10|-0.28773618|
|      2|   4.0|    10| 0.82801116|
|      4|   3.0|    10| 0.37610853|
|      6|   3.0|    24|  0.7564494|
|      4|   1.0|    29| -0.6311784|
|      0|   1.0|    11|  0.1147214|
|      6|   2.0|    11|  0.5672176|
|      4|   3.0|    18|  1.9205147|
|      5|   2.0|    18|  1.7931112|
+-------+------+------+-----------+
only showing top 20 rows



In [0]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.6124548112547465


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [0]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [0]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|      6|    11|
|     10|    11|
|     25|    11|
|     32|    11|
|     38|    11|
|     48|    11|
|     50|    11|
|     51|    11|
|     59|    11|
|     76|    11|
|     89|    11|
|     94|    11|
+-------+------+



In [0]:
reccomendations = model.transform(single_user)

In [0]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     51|    11|  3.9637768|
|     25|    11|  3.2589319|
|     38|    11|  2.8045673|
|     76|    11|  2.1997225|
|     48|    11|  1.1603427|
|     89|    11|   1.016454|
|      6|    11|  0.5672176|
|     32|    11|  0.3944963|
|     59|    11| 0.34800565|
|      0|    11|  0.1147214|
|     10|    11|0.034166306|
|     94|    11| -0.4967351|
|     50|    11| -0.7389121|
+-------+------+-----------+

